In [10]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain.agents.tool_calling_agent.base import create_tool_calling_agent
from langchain.agents.agent import AgentExecutor
from langchain_core.tools import tool
from langchain import hub
load_dotenv(override=True)

True

In [2]:
@tool
def multiply(a: float, b: float) -> float:
    """
    Helps in multiplying two numbers
    """
    return a * b

In [3]:
tools = [TavilySearchResults(), multiply]

In [22]:
prompt = hub.pull("langchain-ai/react-agent-template").partial(instructions = "", 
                                                               tools = tools, 
                                                               tool_names = ", ".join([i.name for i in tools]))

In [23]:
print(prompt.template)

{instructions}

TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}


In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

In [24]:
agent = create_tool_calling_agent(llm = ChatOpenAI(model = "gpt-4o-mini"), 
                                  tools= tools, 
                                  prompt = prompt)

In [25]:
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [26]:
agent_executor.invoke({
    "input": "Multiply 2 and 3",
    "chat_history": []
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 2, 'b': 3}`


6.0```
Thought: Do I need to use a tool? No
Final Answer: The result of multiplying 2 and 3 is 6.
```

> Finished chain.


{'input': 'Multiply 2 and 3',
 'chat_history': [],
 'output': '```\nThought: Do I need to use a tool? No\nFinal Answer: The result of multiplying 2 and 3 is 6.\n```'}

In [27]:
agent_executor.invoke({
    "input": "Give me the weather update of Mumbai and New York"
})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Mumbai weather update'}`


[{'title': 'Rainy Weekend For Mumbai: Heavy Rains Likely - Skymet Weather', 'url': 'https://www.skymetweather.com/content/weather-news-and-analysis/rainy-weekend-for-mumbai-heavy-rains-likely', 'content': '11 जून के बाद फिर से बारिश की उम्मीद है, जिससे राहत और मानसून की प्रगति संभव है। posted on: 11/06/2025 · thumbnail image. Uttar Pradesh Weather Update: Hot and Dry', 'score': 0.86080295}, {'title': 'Mumbai Weather Today (Wednesday, Jun 11, 2025)', 'url': 'https://www.indiatoday.in/weather/mumbai-weather-forecast-today', 'content': '![humdity]( https://cf-img-a-in.tosshub.com/sites/weather/resources/image/humidity.svg)\n![pressure](https://cf-img-a-in.tosshub.com/sites/weather/resources/image/pressure.svg)\n![lon]( https://cf-img-a-in.tosshub.com/sites/weather/resources/image/lon.svg)\n![lat]( https://cf-img-a-in.tosshub.com/sites/weather/resources/image/lat.svg)

{'input': 'Give me the weather update of Mumbai and New York',
 'output': "```\nThought: Do I need to use a tool? No\nFinal Answer: Here are the weather updates:\n\n**Mumbai:**\n- Heavy rains are likely this weekend, and rain is expected after June 11, which may provide relief and indicate the progression of the monsoon.\n  - [Read more here](https://www.skymetweather.com/content/weather-news-and-analysis/rainy-weekend-for-mumbai-heavy-rains-likely)\n\n- Today's weather forecast shows conditions with high humidity, and the city's temperatures are expected to fluctuate.\n  - [More details](https://www.indiatoday.in/weather/mumbai-weather-forecast-today)\n  \n**New York:**\n- New York is experiencing a warm and dry pattern, with precautions advised against overexposure to the sun.\n  - [Forecast updated here](https://weathershogun.com/weather/usa/ny/new-york/3284/june/2025-06-11)\n\n- Current temperatures in New York are around 63°F with overcast conditions and light fog.\n  - [Check the